Below is the list of required files that need to uploaded for this script to run:

- `lr_utils.py`
- `utils.py`
- `/content/diabetes_binary_5050split_health_indicators_BRFSS2015.csv`


In [ ]:
pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 15.7 MB/s eta 0:00:00


In [ ]:
import torch
import tenseal as ts
import pandas as pd
import random
from time import time
import numpy as np

from utils import load_diabetes_data_5050, print_metrics
from lr_utils import LR, train, evaluate_model

In [ ]:
#Load BRFSS dataset with 50/50 split
x_train, x_test, y_train, y_test = load_diabetes_data_5050()

##Training LR Model

In [ ]:
#Define training parameters
n_features = x_train.shape[1]
model = LR(n_features)
optim = torch.optim.SGD(model.parameters(), lr=1)
criterion = torch.nn.BCELoss()

In [ ]:
model = train(model, optim, criterion, x_train, y_train)

Epoch: 1 	Training Loss: 0.874620
Epoch: 2 	Training Loss: 0.592624
Epoch: 3 	Training Loss: 0.558967
Epoch: 4 	Training Loss: 0.544343
Epoch: 5 	Training Loss: 0.535871
Epoch: 6 	Training Loss: 0.530366
Epoch: 7 	Training Loss: 0.526552
Epoch: 8 	Training Loss: 0.523790
Epoch: 9 	Training Loss: 0.521724
Epoch: 10 	Training Loss: 0.520137


In [ ]:
# Evaluate the model
accuracy, precision, recall, f1, confusion  = evaluate_model(model, x_test, y_test)

print_metrics(accuracy, precision, recall, f1, confusion)

Accuracy: 0.7473
Precision: 0.7354
Recall: 0.7738
F1 Score: 0.7541
Confusion Matrix:
 [[5087 1971]
 [1602 5479]]


##Encrypted Evaluation

In [ ]:
class EncryptedLR:

    def __init__(self, torch_lr):
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()

    def forward(self, enc_x):
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

eelr = EncryptedLR(model)

In [ ]:
# parameters
poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]
# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 20
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()

In [ ]:
t_start = time()
enc_x_test = [ts.ckks_vector(ctx_eval, x.tolist()) for x in x_test]
t_end = time()
print(f"Encryption of the test-set took {int(t_end - t_start)} seconds")

Encryption of the test-set took 36 seconds


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def encrypted_evaluation1(model, enc_x_test, y_test):
  t_start = time()
  y_true = []
  y_pred = []

  for enc_x, y in zip(enc_x_test, y_test):
        # encrypted evaluation
        enc_out = model(enc_x)
        # plain comparison
        output = enc_out.decrypt()
        output = torch.tensor(output)
        output = torch.sigmoid(output)
        predicted = output >= 0.5
        y_true.extend(y.view(-1).tolist())
        y_pred.extend(predicted.view(-1).tolist())

  t_end = time()
  print(f"Evaluated test_set of {len(x_test)} entries in {int(t_end - t_start)} seconds")

  # Calculate metrics
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)
  confusion = confusion_matrix(y_true, y_pred)

  return accuracy, precision, recall, f1, confusion


In [ ]:
accuracy, precision, recall, f1, confusion  = encrypted_evaluation1(eelr, enc_x_test, y_test)

print_metrics(accuracy, precision, recall, f1, confusion)

Evaluated test_set of 14139 entries in 74 seconds
Accuracy: 0.7322
Precision: 0.6740
Recall: 0.9013
F1 Score: 0.7712
Confusion Matrix:
 [[3971 3087]
 [ 699 6382]]
